<a href="https://colab.research.google.com/github/victorgrrtj/trabalho_final_puc/blob/main/trabalho_final_predicao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Trabalho Final - Treinamento do Modelo Preditivo

## Importação de Bibliotecas

In [1]:
!cat /etc/os-release

PRETTY_NAME="Ubuntu 22.04.2 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.2 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [2]:
%%sh
sudo su
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -

curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list > /etc/apt/sources.list.d/mssql-release.list

exit
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get install -y msodbcsql18

# optional: for bcp and sqlcmd
sudo ACCEPT_EULA=Y apt-get install -y mssql-tools18
echo 'export PATH="$PATH:/opt/mssql-tools18/bin"' >> ~/.bashrc
source ~/.bashrc
# optional: for unixODBC development headers
sudo apt-get install -y unixodbc-dev

OK


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100   983  100   983    0     0   4168      0 --:--:-- --:--:-- --:--:--  4182
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    88  100    88    0     0    478      0 --:--:-- --:--:-- --:--:--   480


In [3]:
%%sh
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get install -y msodbcsql17
# optional: for bcp and sqlcmd
sudo ACCEPT_EULA=Y apt-get install -y mssql-tools
echo 'export PATH="$PATH:/opt/mssql-tools/bin"' >> ~/.bashrc
source ~/.bashrc
# optional: for unixODBC development headers
sudo apt-get install -y unixodbc-dev

Get:1 https://packages.microsoft.com/ubuntu/22.04/prod jammy InRelease [3,611 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://packages.microsoft.com/ubuntu/22.04/prod jammy/main arm64 Packages [16.7 kB]
Get:7 https://packages.microsoft.com/ubuntu/22.04/prod jammy/main all Packages [970 B]
Get:8 https://packages.microsoft.com/ubuntu/22.04/prod jammy/main amd64 Packages [78.1 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://packages.microsoft.com/ubuntu/22.04/prod jammy/main armhf Packages [8,411 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu

W: https://packages.microsoft.com/ubuntu/22.04/prod/dists/jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling

In [4]:
!sudo apt-get install unixodbc-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unixodbc-dev is already the newest version (2.3.9-5).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


In [5]:
!pip install pyodbc
!pip install chart_studio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.5/343.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 881.1 kB/s eta 0:00:00


In [6]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 3.0 MB/s eta 0:00:00


In [7]:
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.2 MB/s eta 0:00:00


In [8]:
# Importação das bibliotecas
import requests
import unidecode
import pandas as pd
import math
import time
import random
import sqlalchemy
from sqlalchemy import create_engine
import pymysql
import pyodbc
import joblib
import datetime as dt
import pytz
headers = {
    'Authorization': 'Bearer $ACCESS_TOKEN',
}

In [20]:
user = 'adminskyblue'
pw = input('Digite a senha: ')
server = 'skyblue.database.windows.net'
database = 'skyblue'


engine = sqlalchemy.create_engine(
    "mssql+pyodbc://"+user+":"+pw+"@"+server+"/"+database+"?driver=ODBC+Driver+17+for+SQL+Server",
    fast_executemany=True
)

Digite a senha: Adminsky1!


## Sumarização de todas as extrações.

In [63]:
""" Contém looping para todas as páginas.
Retorna um dicionário com as informações de cada imóvel.
Devido à quantidade de páginas, este processo pode demorar mais de 30 minutos."""

headers = {
    'Authorization': 'Bearer $ACCESS_TOKEN',
}

offset = 0
pag_inicial = 'https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset='+str(offset)

response = requests.get(
  pag_inicial,
  headers=headers)
json = response.json()
qtd_pag = round(json['paging']['total']/50)
qtd_itens = json['paging']['total']

print("Foram encontradas {} páginas e {} produtos.".format(qtd_pag, qtd_itens))

offset = 0
url_list = [pag_inicial]
for page in range(qtd_pag):
  offset += 50
  prox_pagina = 'https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset='+str(offset)
  url_list.append(prox_pagina)

extracted_data = []

for index, url in enumerate(url_list):
  response = requests.get(
      url,
      headers=headers)
  print("Codigo de resposta da pagina {}: {}".format(url, response.status_code))

  json = response.json()
  results = json['results']
  time.sleep(random.randint(2,6))

  for item in results:
    product_dict = {}
    product_dict['id'] = item.get('id')
    product_dict['titulo'] = item.get('title')
    product_dict['condicao'] = item.get('condition')
    product_dict['link'] = item.get('permalink')
    product_dict['preco'] = item.get('price')
    for attribute in item.get('attributes'):
      if attribute.get('name') == 'Marca':
        product_dict['marca'] = attribute.get('value_name')
      elif attribute.get('name') == 'Linha':
        product_dict['linha'] = attribute.get('value_name')
      elif attribute.get('name') == 'Modelo':
        product_dict['modelo'] = attribute.get('value_name')
    product_dict['nome_vendedor'] = item.get('seller').get('nickname')
    product_dict['nvl_vendedor'] = item.get('seller').get('seller_reputation').get('level_id')

    vendas_concluidas = item.get('seller').get('seller_reputation').get('transactions').get('completed')
    product_dict['vendas_concluidas'] = vendas_concluidas

    if vendas_concluidas == 0:
      product_dict['cat_vendas'] = '0 vendas'
    elif vendas_concluidas < 11:
      product_dict['cat_vendas'] = '+ 1 vendas'
    elif vendas_concluidas < 101:
      product_dict['cat_vendas'] = '+ 10 vendas'
    elif vendas_concluidas < 1001:
      product_dict['cat_vendas'] = '+ 100 vendas'
    else:
      product_dict['cat_vendas'] = '+ 1.000 vendas'

    qtd_avaliacoes = item.get('seller').get('seller_reputation').get('transactions').get('total')
    product_dict['qtd_avaliacoes'] = qtd_avaliacoes
    avlc_pos = item.get('seller').get('seller_reputation').get('transactions').get('ratings').get('positive')
    product_dict['avlc_pos'] = avlc_pos
    avlc_neut = item.get('seller').get('seller_reputation').get('transactions').get('ratings').get('neutral')
    product_dict['avlc_neut'] = avlc_neut
    avlc_neg = item.get('seller').get('seller_reputation').get('transactions').get('ratings').get('negative')
    product_dict['avlc_neg'] = avlc_neg

    if qtd_avaliacoes == 0:
      product_dict['rating'] = 0
    else:
      product_dict['rating'] = ((qtd_avaliacoes * avlc_pos * 5) + (qtd_avaliacoes * avlc_neut * 3) + (qtd_avaliacoes * avlc_neg * 1)) / qtd_avaliacoes
    extracted_data.append(product_dict)

    if qtd_avaliacoes == 0:
      product_dict['cat_avaliacoes'] = '0 avaliações'
    elif qtd_avaliacoes < 11:
      product_dict['cat_avaliacoes'] = '+ 1 avaliação'
    elif qtd_avaliacoes < 101:
      product_dict['cat_avaliacoes'] = '+ 10 avaliações'
    elif qtd_avaliacoes < 1001:
      product_dict['cat_avaliacoes'] = '+ 100 avaliações'
    else:
      product_dict['cat_avaliacoes'] = '+ 1.000 avaliações'

  print('Finished page ' + str(index + 1))
print('Finished all pages')

Foram encontradas 17 páginas e 827 produtos.
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=0: 200
Finished page 1
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=50: 200
Finished page 2
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=100: 200
Finished page 3
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=150: 200
Finished page 4
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=200: 200
Finished page 5
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=250: 200
Finished page 6
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=300: 200
Finished page 7
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=350: 200
Finish

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 821 entries, 0 to 820
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 821 non-null    object 
 1   titulo             821 non-null    object 
 2   condicao           821 non-null    object 
 3   link               821 non-null    object 
 4   preco              821 non-null    float64
 5   marca              821 non-null    object 
 6   linha              821 non-null    object 
 7   modelo             821 non-null    object 
 8   nome_vendedor      821 non-null    object 
 9   nvl_vendedor       218 non-null    object 
 10  vendas_concluidas  821 non-null    int64  
 11  qtd_avaliacoes     821 non-null    int64  
 12  avlc_pos           821 non-null    float64
 13  avlc_neut          821 non-null    float64
 14  avlc_neg           821 non-null    float64
dtypes: float64(4), int64(2), object(9)
memory usage: 96.3+ KB


In [64]:
df = pd.json_normalize(extracted_data)

In [65]:
def formataCondicao(x):
  if x == 'new':
    return 'Novo'
  else:
    return 'Usado'

def formataAvaliacaoVendedor(x):
  if x == '1_red':
    return 'Péssimo'
  elif x == '2_orange':
    return 'Ruim'
  elif x == '3_yellow':
    return 'Regular'
  elif x == '4_light_green':
    return 'Bom'
  elif x == '5_green':
    return 'Excelente'
  else:
    return 'N/A'

def formataArmazenamento(x):
  if '128' in x:
    return '128 GB'
  elif '256'in x:
    return '256 GB'
  elif '512'in x:
    return '512 GB'
  elif 'tb' or 'Tb' or 'TB'in x:
    return '1 TB'
  else:
    return 'N/A'

In [66]:
df['condicao'] = df['condicao'].apply(formataCondicao)

In [67]:
df['nvl_vendedor'] = df['nvl_vendedor'].apply(formataAvaliacaoVendedor)

In [68]:
df['armazenamento'] = df['titulo'].apply(formataArmazenamento)

In [69]:
df.shape

(827, 19)

In [70]:
df['ts_carga'] = str(dt.datetime.now(pytz.timezone('Etc/GMT+3')).strftime('%d-%m-%Y %X'))

In [71]:
df.drop(df.index[df['modelo'] == 'Pro'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 7'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 7 Plus'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 12 Pro'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 12'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone XR'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone XS'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhones 8 ao 14'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 14 normal'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 11'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 11 Pro'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 11 Pro Max'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 13'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 13 Pro'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 13 Pro Max'], inplace=True)
df.drop(df.index[df['titulo'].str.contains('iphone', case=False) == False], inplace=True)
df.drop(df.index[df['titulo'].str.contains('14', case=False) == False], inplace=True)
df.reset_index(inplace=True, drop=True)

In [72]:
df['titulo'] = df['titulo'].apply(lambda x: unidecode.unidecode(x))

In [43]:
df.head()

,id,titulo,condicao,link,preco,marca,linha,modelo,nome_vendedor,nvl_vendedor,vendas_concluidas,cat_vendas,qtd_avaliacoes,avlc_pos,avlc_neut,avlc_neg,rating,cat_avaliacoes,armazenamento,ts_carga
0,MLB3829572796,Apple iPhone 14 Pro (128 Gb) - Roxo-profundo,Novo,https://www.mercadolivre.com.br/apple-iphone-1...,6571.0,Apple,iPhone,iPhone 14 Pro,NPGAMES01,Excelente,1135,+1000 vendas,1238,0.94,0.00,0.06,4.76,+1000 avaliações,128 GB,24-07-2023 23:54:47
1,MLB3382690563,Apple iPhone 14 (128 Gb) - Meia-noite,Novo,https://www.mercadolivre.com.br/apple-iphone-1...,5078.0,Apple,iPhone,iPhone 14,BALBINO ELETRÔNICOS,Excelente,36,+10 vendas,39,0.00,1.00,0.00,3.00,+10 avaliações,128 GB,24-07-2023 23:54:47
2,MLB3384473755,Apple iPhone 14 Pro (1 Tb) - Prateado,Novo,https://www.mercadolivre.com.br/apple-iphone-1...,10399.0,Apple,iPhone,iPhone 14 Pro,PREÇO.BAIXO.BRASIL,Excelente,6549,+1000 vendas,7764,0.92,0.01,0.07,4.70,+1000 avaliações,1 TB,24-07-2023 23:54:47
3,MLB3340408779,Apple iPhone 14 Pro (256 Gb) - Dourado,Novo,https://www.mercadolivre.com.br/apple-iphone-1...,7640.0,Apple,iPhone,iPhone 14 Pro,GODINHOERICA20230109153505,Regular,485,+100 vendas,637,0.65,0.04,0.31,3.68,+100 avaliações,256 GB,24-07-2023 23:54:47
4,MLB3683852876,Apple iPhone 14 Pro (512 Gb) - Dourado,Novo,https://www.mercadolivre.com.br/apple-iphone-1...,9250.0,Apple,iPhone,iPhone 14 Pro,R9DISTRIBUIDORA,Excelente,1418,+1000 vendas,1638,0.82,0.02,0.16,4.32,+1000 avaliações,512 GB,24-07-2023 23:54:47


In [73]:
model = joblib.load("model.joblib")
cv = joblib.load("cv.joblib")

In [74]:
modelos_features = cv.transform(df['titulo'].values)

df['modelo'] = model.predict(modelos_features)

In [76]:
df.to_sql("vendas2", engine, index=False, chunksize=500, if_exists = "append")

-2